## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
import os
sys.path.append('../')


# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Peniche,PT,55.99,clear sky,39.3558,-9.3811,Hotel Star inn Peniche
1,3,Waiouru,NZ,49.01,scattered clouds,-39.4833,175.6667,Tāwhiri
2,5,Petropavlovsk-Kamchatskiy,RU,30.20,clear sky,53.0452,158.6483,"ООО ""Гостиничный комплекс ""Петропавловск"""
3,7,Qaqortoq,GL,37.40,overcast clouds,60.7167,-46.0333,Hotel Qaqortoq
4,8,Ushuaia,AR,37.40,scattered clouds,-54.8000,-68.3000,Albatros Hotel


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[(vacation_df["Hotel Name"] == "Travel Inn Motel") & (vacation_df["City"]=="Canon City") ].drop_duplicates(subset = ["City","Country","Max Temp","Current Description","Lat","Lng","Hotel Name"])
vacation_end = vacation_df.loc[(vacation_df["Hotel Name"] == "Travel Inn Motel") & (vacation_df["City"]=="Canon City") ].drop_duplicates(subset = ["City","Country","Max Temp","Current Description","Lat","Lng","Hotel Name"])

vacation_stop1 =vacation_df.loc[(vacation_df["Hotel Name"] == "Holiday Inn Express & Suites Salina") & (vacation_df["City"]=="Salina") ].drop_duplicates(subset = ["City","Country","Max Temp","Current Description","Lat","Lng","Hotel Name"])

vacation_stop2 = vacation_df.loc[(vacation_df["Hotel Name"] == "Select Inn Breckenridge") & (vacation_df["City"]=="Wahpeton") ].drop_duplicates(subset = ["City","Country","Max Temp","Current Description","Lat","Lng","Hotel Name"])

vacation_stop3 = vacation_df.loc[(vacation_df["Hotel Name"] == "Six Acres Bed & Breakfast") & (vacation_df["City"]=="Hamilton") ].drop_duplicates(subset = ["City","Country","Max Temp","Current Description","Lat","Lng","Hotel Name"])


In [6]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]


In [7]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
import gmaps.datasets

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
travel_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2,stop3],
        travel_mode='DRIVING')

fig.add_layer(travel_itinerary)
fig


Figure(layout=FigureLayout(height='420px'))

In [8]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
#frames = [start, stop1, stop2,stop3]
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1952,Canon City,US,32.0,snow,38.4410,-105.2424,Travel Inn Motel
1,1489,Salina,US,55.4,overcast clouds,38.8403,-97.6114,Holiday Inn Express & Suites Salina
2,582,Wahpeton,US,35.6,clear sky,46.2652,-96.6059,Select Inn Breckenridge
3,661,Hamilton,US,37.4,clear sky,39.1834,-84.5333,Six Acres Bed & Breakfast


In [9]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [10]:
# 11a. Add a marker layer for each city to the map.
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))